In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from PIL import Image
import os
from io import BytesIO
import logging

In [14]:
path = r"./images_joh/edit/all"
logging.basicConfig(filename='test.log', encoding='utf-8', level=logging.DEBUG, format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s')

In [15]:
def rename_file(filename, foo):
    s = filename.split('.')
    return s[0]+foo+"."+'jpeg'

In [16]:
f = {}
filenames = os.walk(path)
logging.debug("filenames: ")
i = 0
for f_path, _, filename in filenames: 
    for file in filename:
        if file.endswith(".jpeg"):
            logging.debug(f_path+'/'+file)
            f[file] = np.array(Image.open(f_path+"/"+file))
            filename = rename_file(file, "_r")
            f[filename] = np.rot90(np.array(Image.open(f_path+"/"+file)), 2)
            i += 1
            
logging.debug(f"read images #{i}")

In [17]:
# jpg
def generate_jpg(f, sigmas):
    f_jpg = f.copy()
    logging.debug("generating jpgs...")
    i = 0
    for sigma in sigmas:
        for file in f.keys():
            logging.debug(f"generate {file} with quality {int(sigma)}")
            im = f[file].copy()
            buffer = BytesIO()
            Image.fromarray(im).save(buffer , 'jpeg', quality=int(sigma))
            im = Image.open(buffer)
            im = np.array(im)
            filename = rename_file(file, '_'+str(sigma))
            f_jpg[filename] = im
            logging.debug(f"source file: {file}")
            logging.debug(f"dest file:   {filename}")
            i += 1
            
            
    logging.debug(f"generated images #{i}")
    return f_jpg

In [18]:
def generate_sigmas(pre):
    sigmas = np.round(np.exp(pre), 0).astype(np.int32)[::-1]
    return sigmas

In [19]:
def generate_imgs(pre):
    #sigmas = generate_sigmas(pre)
    sigmas = [20, 12, 7, 4]
    size = 1200, 1200
    logging.debug(f"sigmas: {sigmas}")
    sigmas_str = map(str, sigmas)
    sigmas_str = '_'.join(sigmas_str)
    logging.debug(f"sigmas_str: {sigmas_str}")
    path = "./img_out/"+sigmas_str
    if os.path.isdir(path):
        os.rmdir(path)
    os.mkdir(path)
    
    f_jpg = generate_jpg(f.copy(), sigmas)
    logging.debug("saved files")
    i = 0
    for file_name, file in f_jpg.items():
        logging.debug(f'./img_out/{sigmas_str}/'+file_name)
        im = Image.fromarray(file)
        im.thumbnail(size, Image.ANTIALIAS)
        im.save(f'./img_out/{sigmas_str}/'+file_name)
        i += 1
    logging.debug(f"saved images #{i}")

In [20]:
#pres = [np.linspace(0, 1, 5), 
#       np.linspace(0, 2, 5),
#       np.linspace(0, 3, 5),
#       np.linspace(0, 4, 5), 
#       np.linspace(0, 5, 5)]

pres = [1]

for pre in pres:
    logging.debug(f"pre: {pre}")
    generate_imgs(pre)



